In [1]:
import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_excel('/kaggle/input/superstore/US Superstore data.xls')
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


EDA

In [3]:
print("This dataset has {} rows and {} columns".format(df.shape[0],df.shape[1]))

This dataset has 9994 rows and 21 columns


In [4]:
df.isnull().sum()

Row ID           0
Order ID         0
Order Date       0
Ship Date        0
Ship Mode        0
Customer ID      0
Customer Name    0
Segment          0
Country          0
City             0
State            0
Postal Code      0
Region           0
Product ID       0
Category         0
Sub-Category     0
Product Name     0
Sales            0
Quantity         0
Discount         0
Profit           0
dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Row ID         9994 non-null   int64         
 1   Order ID       9994 non-null   object        
 2   Order Date     9994 non-null   datetime64[ns]
 3   Ship Date      9994 non-null   datetime64[ns]
 4   Ship Mode      9994 non-null   object        
 5   Customer ID    9994 non-null   object        
 6   Customer Name  9994 non-null   object        
 7   Segment        9994 non-null   object        
 8   Country        9994 non-null   object        
 9   City           9994 non-null   object        
 10  State          9994 non-null   object        
 11  Postal Code    9994 non-null   int64         
 12  Region         9994 non-null   object        
 13  Product ID     9994 non-null   object        
 14  Category       9994 non-null   object        
 15  Sub-Category   9994 n

In [6]:
df[['Sales','Profit','Discount']].describe()

,Sales,Profit,Discount
count,9994.000000,9994.000000,9994.000000
mean,229.858001,28.656896,0.156203
std,623.245101,234.260108,0.206452
min,0.444000,-6599.978000,0.000000
25%,17.280000,1.728750,0.000000
50%,54.490000,8.666500,0.200000
75%,209.940000,29.364000,0.200000
max,22638.480000,8399.976000,0.800000


In [7]:
# from datetime import datetime
# start = datetime(2015,10,11)
# end = datetime(2016,10,11)
# df[(df['Order Date']>start) &(df['Order Date']<end)].shape
# df.drop('Row ID',inplace=True,axis=1)
df[['Order ID','Ship Mode','Customer Name','Order Date','Product Name']].head(10).style\
        .background_gradient(cmap='RdPu',subset=['Order Date'])

,Order ID,Ship Mode,Customer Name,Order Date,Product Name
0,CA-2016-152156,Second Class,Claire Gute,2016-11-08 00:00:00,Bush Somerset Collection Bookcase
1,CA-2016-152156,Second Class,Claire Gute,2016-11-08 00:00:00,"Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back"
2,CA-2016-138688,Second Class,Darrin Van Huff,2016-06-12 00:00:00,Self-Adhesive Address Labels for Typewriters by Universal
3,US-2015-108966,Standard Class,Sean O'Donnell,2015-10-11 00:00:00,Bretford CR4500 Series Slim Rectangular Table
4,US-2015-108966,Standard Class,Sean O'Donnell,2015-10-11 00:00:00,Eldon Fold 'N Roll Cart System
5,CA-2014-115812,Standard Class,Brosina Hoffman,2014-06-09 00:00:00,"Eldon Expressions Wood and Plastic Desk Accessories, Cherry Wood"
6,CA-2014-115812,Standard Class,Brosina Hoffman,2014-06-09 00:00:00,Newell 322
7,CA-2014-115812,Standard Class,Brosina Hoffman,2014-06-09 00:00:00,Mitel 5320 IP Phone VoIP phone
8,CA-2014-115812,Standard Class,Brosina Hoffman,2014-06-09 00:00:00,DXL Angle-View Binders with Locking Rings by Samsill
9,CA-2014-115812,Standard Class,Brosina Hoffman,2014-06-09 00:00:00,Belkin F5C206VTEL 6 Outlet Surge


It is observed that there are many records on same Order ID which is of same customer who has chosen the same ship mode on same date.

# Analyse the Products at Sub-Category level

In [8]:
fig = px.sunburst(df,path=['Country','Category','Sub-Category'],
                 values='Sales',color='Category',
                 hover_data =['Sales','Quantity','Profit'])
fig.update_layout(height=1000,title_text='Product Categories & Sub-Categories')
fig.show()

Above graphs indicates
* Share of each category & sub-category in Sales
* Which sub-category belongs to which category
* Share of sub-category for each category(click on any category to get detailed view of each sub-category)

In [12]:
temp = df[['State','City','Sales']].groupby(['State','City'])['Sales'].sum().reset_index()
fig = px.treemap(temp,path=['State','City'], values='Sales')
fig.update_layout(height=1000,title='City-wise Sales',)
                 #color_discrete_sequence = px.colors.qualitative.Plotly)
fig.data[0].textinfo = 'label+text+value'
fig.show()

Above graph indicates-
* Which city and state has how much sales
* Size of a tile indicates corresponding sales

### Top 5 products

In [13]:
d = []
for i in df["Product Name"].unique():
    d.append([i,round(df[df['Product Name'] == i]['Sales'].sum(),2)])
temp = pd.DataFrame(d,columns=['Product Name','Sales'])
# temp.reset_index(inplace=True)
# del temp['index']
temp.sort_values('Sales',ascending=False).head().fillna(0).style\
        .background_gradient(cmap='Greens',subset=['Sales'])

,Product Name,Sales
1374,Canon imageCLASS 2200 Advanced Copier,61599.820000
451,Fellowes PB500 Electric Punch Plastic Comb Binding Machine with Manual Bind,27453.380000
1398,Cisco TelePresence System EX90 Videoconferencing Unit,22638.480000
483,HON 5400 Series Task Chairs for Big and Tall,21870.580000
1162,GBC DocuBind TL300 Electric Binding System,19823.480000


$Profit = SP - CP$

$CP = SP - Profit$

$Profit Percentage = Profit/CP * 100$
* Remember it should be per unit of quantity basis but in above expression, whole expression becomes per unit basis

In [14]:
df['Cost'] = df['Sales'] - df['Profit']
df['Profit%'] = df['Profit']/df['Cost']*100
df[['Product Name','Profit%']].sort_values('Profit%',ascending=False).head().fillna(0).style\
        .background_gradient(cmap='Greens',subset=['Profit%'])

,Product Name,Profit%
5850,Sharp 1540cs Digital Laser Copier,100.000000
246,"Adams Telephone Message Book w/Frequently-Called Numbers Space, 400 Messages per Book",100.000000
9837,Alphabetical Labels for Top Tab Filing,100.000000
2180,Alphabetical Labels for Top Tab Filing,100.000000
9039,GBC Ibimaster 500 Manual ProClick Binding System,100.000000


In [15]:
d = []
for i in df['Sub-Category'].unique():
    sales = round(df[df['Sub-Category']==i]['Sales'].sum(),2)
    profit = round(df[df['Sub-Category']==i]['Profit'].sum(),2)
    d.append([i,sales,profit])
temp = pd.DataFrame(d,columns=['Sub-Category','Sales','Profit'])
temp = temp.sort_values('Sales',ascending=True)

fig = go.Figure(data=[go.Bar(name='Sales',x=temp['Sales'],y=temp['Sub-Category'],orientation='h',marker_color = 'green'),
                      go.Bar(name='Profit',x=temp['Profit'],y=temp['Sub-Category'],orientation='h',marker_color = 'navy')])
fig.update_layout(template='simple_white',title='Sales & Profit of each Sub-Category',height=700) #barmode='stack'
fig.show()

#### Region-wise Product Sales

In [16]:
# df[(df['Region']=='South') & (df['Sub-Category']=='Binders')]
fig = go.Figure(data=[go.Bar(name=region,x=df['Sub-Category'],y=df[df['Region']==region]['Sales'],marker_color=color) for region,color in zip(df.Region.unique(),['red','navy','green','brown'])])
fig.update_layout(barmode='group',template='simple_white',title='Region-wise Sub-Category products Sales')
fig.show()

# Analyze the Customers

Segmentation

In [17]:
temp = df['Segment'].unique()
fig = go.Figure(data=go.Bar(x=temp,y=[df[df['Segment']==i]['Sales'].sum() for i in temp]))
fig.update_traces(marker_color='rgb(171,241,255)', marker_line_color='rgb(12,0,335)',
                  marker_line_width=2, opacity=0.6)
fig.update_layout(template='simple_white',title='Sales of each Segment')
fig.show()

Best Customers

In [18]:
temp = df.sort_values('Profit',ascending=False)[['Sales','Profit','Customer Name']].head(10)
fig = go.Figure(data=[go.Bar(name='Profit',x=temp['Profit'],y=temp['Customer Name'],orientation='h',marker_color='navy'),
                      go.Bar(name='Sales',x=temp['Sales'],y=temp['Customer Name'],orientation='h',marker_color='green')])
fig.update_layout(template='simple_white',title='Sales & Profit of 10 Best Customers',barmode='stack',
                 yaxis_categoryorder = 'total ascending')
fig.show()

Frequent Purchase Customer

In [19]:
df['Customer Name'].value_counts().nlargest(10)

William Brown          37
Paul Prost             34
John Lee               34
Matt Abelman           34
Chloris Kastensmidt    32
Seth Vernon            32
Jonathan Doherty       32
Edward Hooks           32
Zuschuss Carroll       31
Emily Phan             31
Name: Customer Name, dtype: int64

# Analyze the Orders & Sales

In [20]:
df['Lead Time'] = (df['Ship Date'] - df['Order Date']).dt.days
df['Week'] = df['Order Date'].dt.week
df['Year'] = df['Order Date'].dt.year
df['Month'] = df['Order Date'].dt.month

In [21]:
week14 = df[df['Year']==2014][['Week','Sales']].groupby(['Week']).sum()
week15 = df[df['Year']==2015][['Week','Sales']].groupby(['Week']).sum()
week16 = df[df['Year']==2016][['Week','Sales']].groupby(['Week']).sum()
week17 = df[df['Year']==2017][['Week','Sales']].groupby(['Week']).sum()
month14 = df[df['Year']==2014][['Month','Sales']].groupby(['Month']).sum()
month15 = df[df['Year']==2015][['Month','Sales']].groupby(['Month']).sum()
month16 = df[df['Year']==2016][['Month','Sales']].groupby(['Month']).sum()
month17 = df[df['Year']==2017][['Month','Sales']].groupby(['Month']).sum()
week14.rename(columns={'Sales':'Sales14'},inplace=True)
week15.rename(columns={'Sales':'Sales15'},inplace=True)
week16.rename(columns={'Sales':'Sales16'},inplace=True)
week17.rename(columns={'Sales':'Sales17'},inplace=True)
month14.rename(columns={'Sales':'Sales14'},inplace=True)
month15.rename(columns={'Sales':'Sales15'},inplace=True)
month16.rename(columns={'Sales':'Sales16'},inplace=True)
month17.rename(columns={'Sales':'Sales17'},inplace=True)
week = week14.join(week15,on='Week').join(week16,on='Week').join(week17,on='Week')
month = month14.join(month15,on='Month').join(month16,on='Month').join(month17,on='Month')
month.head()

,Sales14,Sales15,Sales16,Sales17
Month,,,,
1,14236.895,18174.0756,18542.491,43971.3740
2,4519.892,11951.4110,22978.815,20301.1334
3,55691.009,38726.2520,51715.875,58872.3528
4,28295.345,34195.2085,38750.039,36521.5361
5,23648.287,30131.6865,56987.728,44261.1102


In [22]:
fig = make_subplots(rows=4, cols=1,shared_xaxes=True)
fig.add_trace(go.Scatter(x=week.index, y=week['Sales14'],mode='lines+markers',
                         name='2014',marker_color='blue'), row=1, col=1)
fig.add_trace(go.Scatter(x=week.index, y=week['Sales15'],mode='lines+markers',
                         name='2015',marker_color='green'), row=2, col=1)
fig.add_trace(go.Scatter(x=week.index, y=week['Sales16'],mode='lines+markers',
                         name='2016',marker_color='red'), row=3, col=1)
fig.add_trace(go.Scatter(x=week.index, y=week['Sales17'],mode='lines+markers',
                         name='2017',marker_color='navy'), row=4, col=1)
fig.update_layout(template='simple_white',height = 1000,
                   title='Sales per Week')
fig.show()

In [23]:
fig = make_subplots(rows=4, cols=1,shared_xaxes=True)
fig.add_trace(go.Scatter(x=month.index, y=month['Sales14'],mode='lines+markers',
                         name='2014',marker_color='blue'), row=1, col=1)
fig.add_trace(go.Scatter(x=month.index, y=month['Sales15'],mode='lines+markers',
                         name='2015',marker_color='green'), row=2, col=1)
fig.add_trace(go.Scatter(x=month.index, y=month['Sales16'],mode='lines+markers',
                         name='2016',marker_color='red'), row=3, col=1)
fig.add_trace(go.Scatter(x=month.index, y=month['Sales17'],mode='lines+markers',
                         name='2017',marker_color='navy'), row=4, col=1)
fig.update_layout(template='simple_white',height = 1000,
                   title='Sales per month')
fig.show()

# Analyze the Geography

Most Profitable transaction cities

In [24]:
df[['City','Profit','Profit%']].sort_values('Profit',ascending=False).head(10).style\
    .background_gradient(cmap='Greens',subset=['Profit'])\
    .background_gradient(cmap='RdPu',subset=['Profit%'])

,City,Profit,Profit%
6826,Lafayette,8399.976000,92.307692
8153,Seattle,6719.980800,92.307692
4190,Newark,5039.985600,92.307692
9039,Detroit,4946.370000,100.000000
4098,Minneapolis,4630.475500,96.078431
2623,New York City,3919.988800,53.846154
509,Atlanta,3177.475000,100.000000
8488,Arlington,2799.984000,47.058824
7666,Providence,2591.956800,92.307692
6520,Jackson,2504.221600,85.185185


Most Profitable transaction States

In [25]:
df[['State','Profit','Profit%']].sort_values('Profit',ascending=False).head(10).style\
    .background_gradient(cmap='Greens',subset=['Profit'])\
    .background_gradient(cmap="RdPu",subset=['Profit%'])

,State,Profit,Profit%
6826,Indiana,8399.976000,92.307692
8153,Washington,6719.980800,92.307692
4190,Delaware,5039.985600,92.307692
9039,Michigan,4946.370000,100.000000
4098,Minnesota,4630.475500,96.078431
2623,New York,3919.988800,53.846154
509,Georgia,3177.475000,100.000000
8488,Virginia,2799.984000,47.058824
7666,Rhode Island,2591.956800,92.307692
6520,Michigan,2504.221600,85.185185


Transaction counts in different states

In [26]:
df.groupby(['State'])['Sales'].nunique().sort_values(ascending=False)

State
California              1547
New York                 964
Texas                    860
Pennsylvania             525
Washington               461
Illinois                 461
Ohio                     441
Florida                  357
Michigan                 240
North Carolina           236
Arizona                  216
Virginia                 212
Colorado                 179
Tennessee                177
Georgia                  175
Indiana                  146
Kentucky                 137
Massachusetts            129
New Jersey               129
Oregon                   119
Wisconsin                110
Maryland                 104
Delaware                  95
Minnesota                 86
Connecticut               80
Missouri                  64
Oklahoma                  64
Alabama                   61
Arkansas                  58
Rhode Island              54
Mississippi               53
Utah                      53
Louisiana                 41
South Carolina            40
Nevada  

### Where your customers are?

In [27]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
geolocator = Nominatim(user_agent="my-dashboard")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
 
l = [] 
for address in df.City.unique():
    location = geocode(address)
    l.append([address,location.latitude,location.longitude])
df_city = pd.DataFrame(l,columns=['City','Latitude','Longitude'])
df_city.head()

,City,Latitude,Longitude
0,Henderson,32.182598,-95.789318
1,Los Angeles,34.053691,-118.242766
2,Fort Lauderdale,26.122308,-80.143379
3,Concord,43.207178,-71.537476
4,Seattle,47.603832,-122.330062


In [28]:
geography = pd.merge(df,df_city,on='City',how='left')[['Profit','Sales','City','Latitude','Longitude']]
geography.head()

,Profit,Sales,City,Latitude,Longitude
0,41.9136,261.9600,Henderson,32.182598,-95.789318
1,219.5820,731.9400,Henderson,32.182598,-95.789318
2,6.8714,14.6200,Los Angeles,34.053691,-118.242766
3,-383.0310,957.5775,Fort Lauderdale,26.122308,-80.143379
4,2.5164,22.3680,Fort Lauderdale,26.122308,-80.143379


In [29]:
fig = px.scatter_mapbox(
        geography,
        title='map',
        lat="Latitude",
        lon="Longitude",
        color="Sales",
        size="Sales",
        size_max=80,
        hover_name="Profit",
        hover_data=["City", "Profit", "Sales"],
#         color_continuous_scale='Profit',
    )

fig.layout.update(
        margin={"r": 0, "t": 0, "l": 0, "b": 0},
        height=700,
        # width=700,
        coloraxis_showscale=True,
        mapbox_style='stamen-toner',
        mapbox=dict(center=dict(lat=39.7837304, lon=-100.4458825), zoom=3),
    )

fig.data[0].update(
        hovertemplate="Sales: ₹%{customdata[2]} <br>Profit: ₹%{customdata[1]}<br>City: %{customdata[0]}"
    )
fig.show()

* Check out my web app - https://busidashboard.herokuapp.com/. 
* Reference - https://github.com/yesdeepakmittal/Business_Dashboard